<h1> 1.<b>前置作業</b> </h1>

<h4> 注意：要先載ckiptagger </h4>
<ul>
  <li> <a href="https://github.com/ckiplab/ckiptagger">Github官方文檔</a></li>
  <li> <a href="https://medium.com/@br19920702/%E4%B8%AD%E7%A0%94%E9%99%A2%E9%96%8B%E6%BA%90nlp%E5%A5%97%E4%BB%B6-ckiptagger-%E7%B9%81%E4%B8%AD%E4%B8%8D%E7%B5%90%E5%B7%B4-7822fc4efbf">中文安裝教學</a></li>
</ul>

In [1]:
# rm(list=ls())
for v in dir(): del globals()[v]

In [1]:
import os
os.getcwd()
os.chdir('C:\\Users\\chiaming\\Desktop\\python')

In [2]:
import numpy as np
import pandas as pd
import os
import math
import copy

from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")


C:\Users\chiaming\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chiaming\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chiaming\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chiaming\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
all_small = pd.read_csv('C:\\Users\\chiaming\\Desktop\\python\\士貴\\專案資料.csv',engine='python',encoding='utf-8',header=None,names=['日期','標題','內容'])

In [ ]:
all_small

'\\'

<p style="color:#ffffff"> <b>>預先設置函數 </b></p>
<ul>
  <li style="color:#ffffff">辭典權重</li>

  <li style="color:#ffffff">停止詞、停止詞詞性</li>
</ul>

In [5]:
def construct_dictionary(word_to_weight):
    length_word_weight = {}
    
    for word, weight in word_to_weight.items():

        if not word: continue
        try:
            weight = float(weight)
        except ValueError:
            continue
        length = len(word)
        if length not in length_word_weight:
            length_word_weight[length] = {}
        length_word_weight[length][word] = weight
        
    length_word_weight = sorted(length_word_weight.items())
    
    return length_word_weight

In [19]:
word_to_weight = {
    "台G": 1,
    "台GG": 2,
    "開噴": 2,
    "崩盤": 1,
    "空單": 1,
    "中美": 1,
    "支持": 1,
    "華為": 1,
    "川普": 1,
    "下探": 1,
    "上探": 1,
    "不要": 1,
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'台G': 1.0, '開噴': 2.0, '崩盤': 1.0, '空單': 1.0, '中美': 1.0, '支持': 1.0, '華為': 1.0, '川普': 1.0, '下探': 1.0, '上探': 1.0, '不要': 1.0}), (3, {'台GG': 2.0})]


In [22]:
deliminate = ['COMMACATEGORY','COLONCATEGORY','DASHCATEGORY','ETCCATEGORY','EXCLAMATIONCATEGORY'
              ,'PARENTHESISCATEGORY','PAUSECATEGORY','PERIODCATEGORY','QUESTIONCATEGORY','SEMICOLONCATEGOR'
              ,'SPCHANGECATEGORY','WHITESPACE','DE','SHI','V_2','Caa','T','Neu','P','Caa','Cab','Cba','Cbb','Nes','Nep','Neqa','Neqb'
             ,'Da','Dfa','Dfb']
stopword = ['吧','就','都','也','者','/','跟','╱','又','了','才','得','台積','面板'
            ,'半導體','台G','GG','台GG','蘋概股','晶圓','奈米','製程','貿易戰','中美'
           ,'大盤','2330','台gg','gg','台績','IC產業','IC設計','科技股','那斯達克','納斯達克'
            ,'道瓊','台積電','台績電','GG ADR','GG adr','gg ADR','gg adr']

<h1>2.<b>排除遺失值&選擇資料</b></h1>

<p style="color:#009933"> >>優化點1：該用甚麼字篩出符合台積電的文章  </p>

In [8]:
all_small = all_small.dropna()
GG = all_small[(all_small['內容'].str.contains('台積|面板|半導體|台G|GG|蘋概股|晶圓|奈米|製程|貿易戰|中美|大盤|2330|台gg|台g|gg|台績|IC產業|IC設計|科技股|納斯達克|那斯達克|道瓊')==True) & (all_small['標題'].str.contains('[公告]')==False)]

In [ ]:
all_small[(all_small['內容'].str.contains('川普&跪')==True) & (all_small['標題'].str.contains('[公告]')==False)]

<h1>3.<b>確認迴圈時間</b></h1>

In [9]:
Date = list(set(GG['日期'].value_counts().index)) #抓出時間

In [10]:
Date.sort()       #讓資料照時間排
Date = Date[1:]   #確保資料為5/29-11/6

In [24]:
7 and 6

6

<h1>4.<b>實作tf</b></h1>

<p style="color:#009933">  >>優化點2：斷詞能否更準 (到最後他會存成很多csv，去預設資料夾看他切出的詞與分數，是還不錯但仍不好) </p>
<p style="color:#009933">  >>優化點3：tf-idf算法是否仍可改良？ 會不會導致重要的字反而分數較低？例如「開噴」，若每筆文章都提到，代表明天股價應該會下降，但idf會認為他在不同文章出現次數過高，反而使分數下降 </p>
<p style="color:#009933">  >>優化點4：我是以"天"為單位，把所有文章總和起來，給每天一個各詞的tf-idf值，所以和tf-idf原先的目的(找出能夠代表這篇文章的重要詞彙)並不相同 </p>

In [ ]:
#double list
com = [[a+'('+b+')' for a,b in zip(i,j)] for i,j in zip(ws_results, pos_results)]
com

In [21]:
from ckiptagger import WS, POS, NER
import time
import copy


for date in Date[0:3]:
    tStart = time.time()#計時開始
    print('【'+date+'】')
    
    #第一部分： 【斷詞+計算當天詞集+計算當天出現詞數】
    ### 詞數 v.s 詞集:  詞數可以重複，詞集不行  (例如 1th、3th文章都有 "台積" ， 那詞數會是 2個 、詞集會是 1個
    ws_results = ws(GG[GG['日期']==date]['內容'],sentence_segmentation=False,recommend_dictionary = dictionary)
    pos_results = pos(ws_results)
  #  result_com = [[a+'('+b+') 'for a,b in zip(k,v)]for k,v in zip(ws_results,pos_results)]
    word_list = [] #出現詞數 (總和)
    word_set = []  #當天詞集 (單一)
    word_adj_only = []  #當天詞性 (單一)
    word_adj_all = []   #當天詞性 (總和)
    word_com_only = [] #單一總和
    word_com_all = []  #全部次數
    word_com_like = []  #合併ws,pos
    today_num = len(ws_results) #當天文章數輛 (長度)
    
    for counter in range(today_num):
        zip_all = zip(ws_results[counter],pos_results[counter])
        word_temporary = []
        for word,adj in zip_all:
            if (adj not in deliminate and word not in stopword):
                word_list.append(word.strip())
                word_adj_all.append(adj.strip())
                word_temporary.append(word.strip()+'('+adj.strip()+')')
            else:
                continue
        word_com_like.append(word_temporary)
        
    for index in word_com_like:
        for final_count in index:
            word_com_all.append(final_count)
  #  for article_nth in ws_results:
  #      for word in article_nth :    
  #          word_list.append(word)
    word_set = list(set(word_list))
    word_adj_only = list(set(word_adj_all))
    word_com_only = list(set(word_com_all))
    print('>> 共有 ' + str(today_num) + '文章'+ ' / ' + str(len(word_com_only)) + '詞' + ' / ' + str(len(word_com_all)) + '詞數')

    #第二部分： 【創建空的Dataframe，欄位是當天有幾篇文章，列是當天的詞集數量】
    zero_data = np.zeros(shape=(len(word_com_only),today_num))
    globals()['columns'] = []               
    for col in range(today_num):
        columns.append(str(col+1)+'th')
    total = pd.DataFrame(zero_data,index=word_com_only,columns=columns)
    check_word = list(total.index)
    
    ##算次數
    ###########################################################################################
    ###########################################################################################
    for total_col in range(today_num):
        for text_pos in word_com_like[total_col]:
            if text_pos  in check_word:
                total.iloc[check_word.index(text_pos),total_col] +=1
            else:
                continue
    print('>> 次數Dataframe完成')
    ###########################################################################################
    ###########################################################################################
    
    #深複製tf - idf (避免一般複製的標籤行為)
    tf = copy.deepcopy(total)
    idf = copy.deepcopy(total)
    
    #tf矩陣計算
    for tf_col in range(len(tf.columns)):
        col_sum = tf.sum(axis=0)[tf_col]
        for tf_row in range(len(tf)):
            if (tf.iloc[tf_row,tf_col] != 0):
                tf.iloc[tf_row,tf_col] = round(tf.iloc[tf_row,tf_col]/col_sum,6)
    print('>> tf矩陣完成')
    
    #idf矩陣成形
    for idf_col in range(len(idf.columns)):
        for idf_text_rol in idf.iloc[np.where(idf.iloc[:,idf_col]!=0)[0],:].index:
            idf.iloc[np.where(idf.index==idf_text_rol)[0],idf_col]=(math.log(len(ws_results)/len(np.where(idf.loc[idf_text_rol]!=0)[0])))
    print('>> idf矩陣完成')
    
    #tf idf互乘
    tfidf = pd.DataFrame(np.array(tf)*np.array(idf),columns=columns,index=word_com_only)
    print('>> tf-idf矩陣完成')
    
    #給出每天的 詞-tfidf分數 資料表
    tfidf_text = pd.DataFrame(tfidf.mean(axis=1),columns=['分數']) #列平均
    word = copy.deepcopy(tfidf_text.sort_values('分數',ascending=False).index)
    score = copy.deepcopy(tfidf_text.sort_values('分數',ascending=False)['分數'])
    pd.DataFrame({'詞':word,'分數':score}).to_csv(date+'.csv',encoding='utf_8_sig',index=False)
    tEnd = time.time()#計時結束
    print('>>>>>> 結束並存檔 (共' + str(round(tEnd-tStart,2)) + '秒)')

【2019-05-29】
>> 共有 6文章 / 181詞 / 235詞數
>> 次數Dataframe完成
>> tf矩陣完成
>> idf矩陣完成
>> tf-idf矩陣完成
>>>>>> 結束並存檔 (共5.03秒)
【2019-05-30】
>> 共有 246文章 / 3834詞 / 10942詞數
>> 次數Dataframe完成
>> tf矩陣完成
>> idf矩陣完成
>> tf-idf矩陣完成
>>>>>> 結束並存檔 (共125.87秒)
【2019-05-31】
>> 共有 217文章 / 3267詞 / 9997詞數
>> 次數Dataframe完成
>> tf矩陣完成
>> idf矩陣完成
>> tf-idf矩陣完成
>>>>>> 結束並存檔 (共219.09秒)
